<a href="https://colab.research.google.com/github/LuscasD/ArTest_LucasRyan/blob/main/Teste_de_Criar_videos_com_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y xvfb python-opengl ffmpeg
!pip install gymnasium[classic_control] "stable-baselines3>=2.0.0" pyvirtualdisplay imageio imageio-ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-opengl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.5 MB/s eta 0:00:00


In [ ]:
from pyvirtualdisplay import Display
from IPython.display import Video

In [ ]:
import gymnasium as gym, imageio

In [ ]:
import torch, torch.nn as nn, torch.optim as optim

In [ ]:
import numpy as np

In [ ]:
env = gym.make("Acrobot-v1")
obs_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

policy = nn.Sequential(
    nn.Linear(obs_dim, 32), nn.ReLU(),
    nn.Linear(32, n_actions), nn.Softmax(dim=-1)
)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def run_episode():
    obs, _ = env.reset()
    log_probs, rewards = [], []
    done = False
    while not done:
        obs_t = torch.tensor(obs, dtype=torch.float32)
        probs = policy(obs_t)
        dist = torch.distributions.Categorical(probs)
        action = dist.sample()
        obs, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated
        log_probs.append(dist.log_prob(action))
        rewards.append(reward)
    return log_probs, rewards

def reinforce(epochs=200):
    for epoch in range(epochs):
        log_probs, rewards = run_episode()
        returns = []
        G = 0
        for r in reversed(rewards):
            G = r + 0.99 * G
            returns.insert(0, G)
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        loss = -torch.sum(torch.stack(log_probs) * returns)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, total reward = {sum(rewards)}")

reinforce(epochs=200)

Epoch 0, total reward = -472.0
Epoch 10, total reward = -379.0
Epoch 20, total reward = -308.0
Epoch 30, total reward = -229.0
Epoch 40, total reward = -225.0
Epoch 50, total reward = -184.0
Epoch 60, total reward = -111.0
Epoch 70, total reward = -115.0
Epoch 80, total reward = -101.0
Epoch 90, total reward = -96.0
Epoch 100, total reward = -109.0
Epoch 110, total reward = -145.0
Epoch 120, total reward = -118.0
Epoch 130, total reward = -241.0
Epoch 140, total reward = -84.0
Epoch 150, total reward = -89.0
Epoch 160, total reward = -92.0
Epoch 170, total reward = -110.0
Epoch 180, total reward = -118.0
Epoch 190, total reward = -102.0


In [ ]:
# Importando as bibliotecas necessárias para construir a rede neural
import torch
import torch.nn as nn
import torch.optim as optim

# Definindo a classe para a rede neural da DQN
class DQN(nn.Module):
    def __init__(self, obs_dim, n_actions):
        # Chamando o construtor da classe pai (nn.Module)
        super(DQN, self).__init__()
        # Definindo as camadas da rede neural
        # Aumentamos o número de neurônios para 64 (dobro de 32) e adicionamos mais uma camada oculta
        self.fc1 = nn.Linear(obs_dim, 64) # Primeira camada totalmente conectada
        self.fc2 = nn.Linear(64, 64)      # Segunda camada totalmente conectada (camada adicional)
        self.fc3 = nn.Linear(64, n_actions) # Camada de saída, com o número de neurônios igual ao número de ações possíveis

    def forward(self, x):
        # Definindo a passagem forward dos dados pela rede
        x = torch.relu(self.fc1(x)) # Aplicando a função de ativação ReLU na saída da primeira camada
        x = torch.relu(self.fc2(x)) # Aplicando a função de ativação ReLU na saída da segunda camada
        return self.fc3(x)          # Retornando a saída da camada final (Q-values para cada ação)

# Exemplo de como instanciar a DQN (usando as dimensões do ambiente Acrobot-v1)
# obs_dim e n_actions já foram definidos nas células anteriores
dqn_policy = DQN(obs_dim, n_actions)

# Definindo o otimizador para a DQN
dqn_optimizer = optim.Adam(dqn_policy.parameters(), lr=1e-3) # Usando Adam como otimizador com taxa de aprendizado 1e-3

print("Modelo DQN com o dobro de neurônios e camadas criado:")
print(dqn_policy)

In [ ]:
from pyvirtualdisplay import Display
import imageio
from IPython.display import Video

display = Display(visible=False, size=(640,480))
display.start()

env = gym.make("Acrobot-v1", render_mode="rgb_array")
obs, _ = env.reset()
frames = []

done = False
while not done:
    obs_t = torch.tensor(obs, dtype=torch.float32)
    with torch.no_grad():
        probs = policy(obs_t)
        action = torch.argmax(probs).item()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    frames.append(env.render())

env.close()
imageio.mimsave("reinforce_cartpole.mp4", frames, fps=30)
Video("reinforce_cartpole.mp4", embed=True)

## Instalar as dependências do box2d


Certificar-se de que as bibliotecas necessárias para os ambientes Box2D estão instaladas.


**Reasoning**:
Install the necessary packages for Box2D environments.



In [ ]:
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (8,996 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

## Criar o ambiente box2d

Instanciar um ambiente específico do Box2D, como 'LunarLander-v2'.


In [ ]:
env_box2d = gym.make("LunarLander-v2")
obs_dim_box2d = env_box2d.observation_space.shape[0]
n_actions_box2d = env_box2d.action_space.n

print(f"Observation space dimension: {obs_dim_box2d}")
print(f"Number of actions: {n_actions_box2d}")

/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment LunarLander-v2 is out of date. You should consider upgrading to version `v3`.
  logger.deprecation(


DeprecatedEnv: Environment version v2 for `LunarLander` is deprecated. Please use `LunarLander-v3` instead.

In [ ]:
env_box2d = gym.make("LunarLander-v3")
obs_dim_box2d = env_box2d.observation_space.shape[0]
n_actions_box2d = env_box2d.action_space.n

print(f"Observation space dimension: {obs_dim_box2d}")
print(f"Number of actions: {n_actions_box2d}")

Observation space dimension: 8
Number of actions: 4


## Definir a rede neural da dqn para o ambiente box2d

Adaptar a arquitetura da rede neural da DQN para as dimensões de observação e ação do ambiente Box2D escolhido.


In [ ]:
# Definindo a classe para a rede neural da DQN para ambientes Box2D
class DQNBox2D(nn.Module):
    def __init__(self, obs_dim, n_actions):
        super(DQNBox2D, self).__init__()
        # Ajustando as camadas para o número de neurônios especificado (64)
        self.fc1 = nn.Linear(obs_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, n_actions)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Instanciando a DQN para o ambiente Box2D (LunarLander-v3)
dqn_box2d_policy = DQNBox2D(obs_dim_box2d, n_actions_box2d)

# Definindo o otimizador para a nova rede DQN
dqn_box2d_optimizer = optim.Adam(dqn_box2d_policy.parameters(), lr=1e-3)

print("Modelo DQN para Box2D com o dobro de neurônios e camadas criado:")
print(dqn_box2d_policy)

Modelo DQN para Box2D com o dobro de neurônios e camadas criado:
DQNBox2D(
  (fc1): Linear(in_features=8, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)


## Configurar o treinamento da dqn

Definir os parâmetros de treinamento específicos para o ambiente Box2D (taxa de aprendizado, fator de desconto, e-greedy policy, etc.).


In [ ]:
gamma = 0.99
episodes = 1000
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
replay_buffer_size = 10000
batch_size = 64

print(f"Training parameters defined:")
print(f"  Gamma: {gamma}")
print(f"  Episodes: {episodes}")
print(f"  Epsilon Start: {epsilon_start}")
print(f"  Epsilon End: {epsilon_end}")
print(f"  Epsilon Decay: {epsilon_decay}")
print(f"  Replay Buffer Size: {replay_buffer_size}")
print(f"  Batch Size: {batch_size}")

Training parameters defined:
  Gamma: 0.99
  Episodes: 1000
  Epsilon Start: 1.0
  Epsilon End: 0.01
  Epsilon Decay: 0.995
  Replay Buffer Size: 10000
  Batch Size: 64


## Implementar o loop de treinamento da dqn


Criar o loop principal de treinamento, incluindo a interação com o ambiente, o armazenamento de experiências (replay buffer), o cálculo da função de perda e a otimização da rede.


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random
from collections import deque

# Hiperparâmetros (ajuste conforme necessário)
tamanho_replay_buffer = 10000
tamanho_batch = 64
gamma = 0.99
epsilon_inicial = 1.0
epsilon_minimo = 0.01
taxa_decadencia_epsilon = 0.995
episodios = 1000

# Inicialização
replay_buffer = deque(maxlen=tamanho_replay_buffer)
epsilon = epsilon_inicial

for episodio in range(episodios):
    observacao, _ = env_box2d.reset()
    terminado = False
    recompensa_total = 0

    while not terminado:
        # Estratégia epsilon-greedy
        if random.random() < epsilon:
            acao = env_box2d.action_space.sample()  # ação aleatória (exploração)
        else:
            obs_tensor = torch.tensor(observacao, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                q_valores = dqn_box2d_policy(obs_tensor)
            acao = torch.argmax(q_valores).item()  # melhor ação (exploração controlada)

        # Executa ação no ambiente
        proxima_obs, recompensa, feito, truncado, _ = env_box2d.step(acao)
        terminado = feito or truncado
        recompensa_total += recompensa

        # Armazena experiência no replay buffer
        replay_buffer.append((observacao, acao, recompensa, proxima_obs, terminado))

        # Treinamento se houver amostras suficientes
        if len(replay_buffer) >= tamanho_batch:
            lote = random.sample(replay_buffer, tamanho_batch)
            obs_lote, acao_lote, recompensa_lote, prox_obs_lote, feito_lote = zip(*lote)

            # Converte tudo para tensores PyTorch
            obs_lote = torch.tensor(np.array(obs_lote), dtype=torch.float32)
            acao_lote = torch.tensor(np.array(acao_lote), dtype=torch.int64)
            recompensa_lote = torch.tensor(np.array(recompensa_lote), dtype=torch.float32)
            prox_obs_lote = torch.tensor(np.array(prox_obs_lote), dtype=torch.float32)
            feito_lote = torch.tensor(np.array(feito_lote), dtype=torch.float32)

            # Calcula Q-alvo (target Q-values)
            with torch.no_grad():
                prox_q_valores = dqn_box2d_policy(prox_obs_lote).max(1)[0]
                q_alvo = recompensa_lote + gamma * prox_q_valores * (1 - feito_lote)

            # Calcula Q-atual (current Q-values)
            q_atual = dqn_box2d_policy(obs_lote).gather(1, acao_lote.unsqueeze(-1)).squeeze(-1)

            # Calcula e aplica o erro (loss)
            perda = nn.MSELoss()(q_atual, q_alvo)
            dqn_box2d_optimizer.zero_grad()
            perda.backward()
            dqn_box2d_optimizer.step()

        # Atualiza a observação
        observacao = proxima_obs

    # Decai o epsilon (menos aleatório com o tempo)
    epsilon = max(epsilon_minimo, epsilon * taxa_decadencia_epsilon)

    #  Exibe progresso
    if (episodio + 1) % 100 == 0 or episodio == 0:
        print(f"Episódio {episodio + 1} | Recompensa Total: {recompensa_total:.2f} | Epsilon: {epsilon:.3f}")


Episódio 1 | Recompensa Total: -291.81 | Epsilon: 0.995
Episódio 100 | Recompensa Total: -203.95 | Epsilon: 0.606
Episódio 200 | Recompensa Total: -377.56 | Epsilon: 0.367
Episódio 300 | Recompensa Total: -2026.03 | Epsilon: 0.222
Episódio 400 | Recompensa Total: -146.99 | Epsilon: 0.135
Episódio 500 | Recompensa Total: -332.70 | Epsilon: 0.082
Episódio 600 | Recompensa Total: -660.26 | Epsilon: 0.049
Episódio 700 | Recompensa Total: -435.14 | Epsilon: 0.030
Episódio 800 | Recompensa Total: -1312.68 | Epsilon: 0.018
Episódio 900 | Recompensa Total: -419.25 | Epsilon: 0.011


KeyboardInterrupt: 

## Salvar o vídeo de saída e avaliar a DQN

Configurar o ambiente para renderizar e salvar um vídeo durante a avaliação do agente treinado, e executar o agente para gerar o vídeo.

In [ ]:
# Configurar a exibição virtual para renderização
display = Display(visible=False, size=(600, 400))
display.start()

# Criar o ambiente com renderização para vídeo
eval_env = gym.make("LunarLander-v3", render_mode="rgb_array")

# Lista para armazenar os frames do vídeo
frames = []

# Resetar o ambiente e obter a observação inicial
obs, _ = eval_env.reset()
done = False

# Loop de avaliação para gerar o vídeo
while not done:
    # Selecionar a ação usando a política treinada (sem exploração epsilon-greedy)
    obs_t = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        q_values = dqn_box2d_policy(obs_t)
    action = torch.argmax(q_values).item()

    # Executar a ação no ambiente
    next_obs, reward, terminated, truncated, _ = eval_env.step(action)
    done = terminated or truncated

    # Adicionar o frame atual à lista
    frames.append(eval_env.render())

    # Atualizar a observação
    obs = next_obs

# Fechar o ambiente de avaliação
eval_env.close()

# Salvar os frames como um arquivo de vídeo
video_path = "dqn_lunarlander.mp4"
imageio.mimsave(video_path, frames, fps=30)

# Exibir o vídeo no notebook
print(f"Vídeo salvo em: {video_path}")
Video(video_path, embed=True)

Vídeo salvo em: dqn_lunarlander.mp4
